# Matrix Infection
You are given a matrix where each cell is either:
- 0: Empty
- 1: Uninfected
- 2: Infected

With each passing second, every infected cell (2) infects its uninfected neighboring cells (1) that are 4-directionally adjacent. Determine the number of seconds required for all uninfected cells to become infected. If this is impossible, return ‐1.

```python
Input: matrix = [[1, 1, 1, 0], [0, 0, 2, 1], [0, 1, 1, 0]]
Output: 3
```

## Intuition
Let's begin tackling this problem by considering a simple case where the initial matrix contains only one infected cell.

---

### Matrix with One Infected Cell
Consider a matrix with just one infected cell. An important observation is that each infected (2) and uninfected (1) cell can be treated as nodes in a graph, where edges exist between cells that are 4-directionally adjacent. Therefore, we can visualize these cells as a connected graph.

This perspective allows us to frame the problem as a graph traversal problem. But which traversal algorithm best simulates the infection process? To answer this, let's analyze how cells get infected over time.

- After the first second, the adjacent uninfected neighbors of the initially infected cell become infected. These cells are at a distance of 1 from the original infected cell.
- One second later, the neighbors of these newly infected cells become infected. These cells are at a distance of 2 from the initial infected cell.

From this pattern, we see that the outward expansion of the infection resembles level-order traversal in a tree, where each level represents nodes at a specific distance from the initially infected node.

Thus, we should use a level-order traversal approach to spread the infection, starting from the infected cell. Each level we traverse corresponds to one second passing in the infection process.

For implementation, we know that level-order traversal is a modified version of **Breadth-First Search (BFS)**, so we use a **queue** to implement this traversal.

---

### Matrix with Multiple Infected Cells
If there are multiple initially infected cells, meaning multiple cells exist at level 0 of the traversal, we use a pattern known as **multi-source BFS**. Instead of adding just one cell to the queue before starting the traversal, we enqueue every initially infected cell. This way, the traversal starts with all infected cells at level 0, allowing the infection to spread simultaneously from multiple starting points.

---

### Unreachable Uninfected Cells
It's important to note that it's not always possible to infect all uninfected cells. There may be cases where certain uninfected cells are unreachable.

To handle this scenario, we need to check if any uninfected cells remain after the level-order traversal. One way to do this is by scanning the matrix at the end of the traversal to check for remaining `1`s. However, a more efficient approach is:

1. While identifying the level 0 infected cells, also count the total number of uninfected (`1`) cells.
2. As the level-order traversal progresses, decrement this count for each newly infected cell.
3. If, after the traversal, this count is greater than 0, it means some cells remained uninfected. In this case, we return `-1`.

This ensures a cleaner and more efficient way to determine if all uninfected cells have been reached.

In [1]:
from typing import List
from collections import deque

def matrix_infection(matrix: List[List[int]]) -> int:
    dirs = [(-1, 0), (0, -1), (1, 0), (0, 1)]
    queue = deque()
    ones = seconds = 0

    for r in range(len(matrix)):
        for c in range(len(matrix[0])):
            
            if matrix[r][c] == 1:
                ones += 1
            elif matrix[r][c] == 2:
                queue.append((r, c))
    
    while queue and ones > 0:
        seconds += 1

        for _ in range(len(queue)):
            r, c = queue.popleft()

            for d in dirs:
                next_r, next_c = r + d[0], c + d[1]
                if (is_within_bounds(next_r, next_c, matrix) and matrix[next_r][next_c] == 1):
                    matrix[next_r][next_c] = 2
                    ones -= 1
                    queue.append((next_r, next_c))

    return seconds if ones == 0 else -1

def is_within_bounds(r: int, c: int, matrix: List[List[int]]) -> bool:
    return 0 <= r < len(matrix) and 0 <= c < len(matrix[0])

## Complexity Analysis

### Time complexity
The time complexity is O(m*n), where m denotes the number of rows and n denotes the number of columns. This is because in the worst case, every cell in the matrix is explored during level-order traversal.

---

### Space complexity
The space complexity is O(m\*n), primarily due to the queue, which can store up to m\*n cells.